# Final project

Written for Python 2

In [613]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

# Counting/transforming
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Loading and labeling the data

In [614]:
# load data
path = '~/AI/final_project/data/'
comments = pd.read_csv(path + 'attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv(path + 'attack_annotations.tsv',  sep = '\t')

### Find attack rating threshold for split

**Final threshold: > 0.5 mean rating**

In [615]:
# group by mean score, then read comments to determine an appropriate threshold for what is an "attack" 
meanattack = annotations.groupby('rev_id')['attack'].mean()
comments['attack_score'] = meanattack
pd.set_option('display.max_colwidth', -1)
comments.query('attack_score > 0.3')[['comment', 'attack_score']].head(30)

comment  \
rev_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [616]:
# labels a comment as an atack if the majority of annotators did so
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# join labels and comments
comments['attack'] = labels

### Check if other comment features are useful

(Useful meaning predictive of attack/not attack)

* **Useful features:** 'logged_in', 'ns', 'sample'
* **Not useful:** 'year'

In [617]:
# Comparing 'logged_in'
print "Attacks:\n",comments.query('attack')['logged_in'].value_counts(),"\n"
print "Not attacks:\n",comments.query('not attack')['logged_in'].value_counts()

Attacks:
False    7635
True     5955
Name: logged_in, dtype: int64 

Not attacks:
True     78963
False    23311
Name: logged_in, dtype: int64


In [618]:
# Comparing 'year'
print "Attacks:\n",comments.query('attack')['year'].value_counts(),"\n"
print "Not attacks:\n",comments.query('not attack')['year'].value_counts()

Attacks:
2008    2239
2006    1997
2007    1894
2009    1683
2010    1301
2011    1035
2012    824 
2015    775 
2014    688 
2013    661 
2005    445 
2016    36  
2004    11  
2003    1   
Name: year, dtype: int64 

Not attacks:
2006    15379
2007    14543
2008    14402
2009    10891
2010    9440 
2011    7609 
2012    6727 
2014    5967 
2015    5922 
2013    5788 
2005    4520 
2004    629  
2016    255  
2003    147  
2002    48   
2001    7    
Name: year, dtype: int64


In [619]:
# Comparing 'ns'
print "Attacks:\n",comments.query('attack')['ns'].value_counts(),"\n"
print "Not attacks:\n",comments.query('not attack')['ns'].value_counts()

Attacks:
user       11341
article    2249 
Name: ns, dtype: int64 

Not attacks:
user       53206
article    49068
Name: ns, dtype: int64


In [620]:
# Comparing 'sample'
print "Attacks:\n",comments.query('attack')['sample'].value_counts(),"\n"
print "Not attacks:\n",comments.query('not attack')['sample'].value_counts()

Attacks:
blocked    13246
random     344  
Name: sample, dtype: int64 

Not attacks:
blocked    65126
random     37148
Name: sample, dtype: int64


In [621]:
# looking at size of splits
comments['split'].value_counts()

train    69526
test     23178
dev      23160
Name: split, dtype: int64

# Cleaning the dataset

### Methods included:
* removing newlines and tabs
* removing punctuation
* differentiating between punctuation that mostly occurs in the middle of a word vs. at the end
* converting all text to lowercase

### Methods tried, but not included:
* excluding stop words
* ngrams set to ‘char_wb’ instead of ‘word’

In [622]:
# remove newline and tab tokens
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

# replace with space
punctuation = ":`.=,-;?!)(&}{[]|<>"
for char in punctuation:
    comments['comment'] = comments['comment'].apply(lambda x: x.replace(char, " "))  

# replace with nothing (bc they generally appear in the middle of a word, e.g., you're -> youre)
punctuation = "*'@#$%"
for char in punctuation:
    comments['comment'] = comments['comment'].apply(lambda x: x.replace(char, ""))
    
comments['comment'] = comments['comment'].apply(lambda x: x.lower())

In [623]:
# check that data looks cleaned
comments

comment  \
rev_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [624]:
# combine all features for classifier later
comments['stats'] = comments.apply(lambda row: {'logged_in': str(row['logged_in']), 
                                                 'ns': row['ns'],
                                                 'sample': row['sample']}, axis=1)

# Classifying

### Models:
* **Logistic regression**
  * Best results (AUC): 0.960
  * Percentage points improved: 0.733%
  * Model with best results prior to tuning 
  
* **Naive Bayes'**
  * Best results (AUC): 0.938
  * Percentage points improved: 3.62%
  
* **Linear SVM**
  * Best results (AUC): 0.959
  * Percentage points improved: 0.306%
  * Best results after tuning
  
* **Stochastic gradient descent (SGD)**
  * Best results (AUC): 0.955
  * Percentage points improved: 2.96%

* **Random forest**
  * Best results (AUC): 0.941
  * Percentage points improved: -0.043%

### Helper functions

In [625]:
def print_top10_features(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    top10 = np.argsort(clf.coef_[0])[-20:]
    print("%s:\n%s" % ("most attack",
            " \n".join(feature_names[j] for j in top10)))
    top10nice = np.argsort(clf.coef_[0])[:20]
    print("\n\n%s:\n%s" % ("least attack",
            " \n".join(feature_names[j] for j in top10nice)))

In [626]:
def print_classification_metrics(probabilities):
    """Prints AUC, accuracy, confusion matrix, precision, recall, fscore, and support"""
    auc = roc_auc_score(test_comments['attack'], probabilities)
    predicted = clf.predict(test_comments['comment'])
    
    print "AUC: %.3f" %auc
    print "Accuracy: %(acc).3f" %{"acc": (100*accuracy_score(test_comments['attack'],predicted))} + "%","\n"
    print "Confusion matrix:\n",confusion_matrix(test_comments['attack'],predicted), "\n"
    print "Other stats:\n", classification_report(test_comments['attack'],predicted,target_names = ['Polite', 'Attack'])

In [627]:
# split dataset into train, dev, and test
train_comments = comments.query("split=='train'")
dev_comments = comments.query("split=='dev'")
test_comments = comments.query("split=='test'")

## Logistic regression

In [628]:
# With default values

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.predict_proba(test_comments['comment'])[:, 1])

AUC: 0.958
Accuracy: 93.891% 

Confusion matrix:
[[20276   146]
 [ 1270  1486]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.94      0.99      0.97     20422
     Attack       0.91      0.54      0.68      2756

avg / total       0.94      0.94      0.93     23178



In [629]:
# Best results

clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000, min_df=10)),
    ('tfidf', TfidfTransformer(norm='l2', sublinear_tf=True)),
    ('clf', LogisticRegression(penalty='l1')),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.predict_proba(test_comments['comment'])[:, 1])

AUC: 0.960
Accuracy: 94.624% 

Confusion matrix:
[[20217   205]
 [ 1041  1715]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.95      0.99      0.97     20422
     Attack       0.89      0.62      0.73      2756

avg / total       0.94      0.95      0.94     23178



## Naive Bayes'

In [630]:
# With default values

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.predict_proba(test_comments['comment'])[:, 1])

AUC: 0.834
Accuracy: 89.805% 

Confusion matrix:
[[20415     7]
 [ 2356   400]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.90      1.00      0.95     20422
     Attack       0.98      0.15      0.25      2756

avg / total       0.91      0.90      0.86     23178



In [631]:
# Best results

clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000, ngram_range = (1,2),min_df=15)),
    ('tfidf', TfidfTransformer(norm='l2', sublinear_tf=True)),
    ('clf', MultinomialNB()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.predict_proba(test_comments['comment'])[:, 1])

AUC: 0.938
Accuracy: 93.425% 

Confusion matrix:
[[20221   201]
 [ 1323  1433]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.94      0.99      0.96     20422
     Attack       0.88      0.52      0.65      2756

avg / total       0.93      0.93      0.93     23178



## Linear SVM

In [632]:
# With default values

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.decision_function(test_comments['comment']))

AUC: 0.953
Accuracy: 94.728% 

Confusion matrix:
[[20170   252]
 [  970  1786]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.95      0.99      0.97     20422
     Attack       0.88      0.65      0.75      2756

avg / total       0.94      0.95      0.94     23178



In [633]:
# Best results

clf = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,2))),
    ('tfidf', TfidfTransformer(norm='l2', sublinear_tf=True)),
    ('clf', LinearSVC(penalty='l1',dual=False, max_iter=500)),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.decision_function(test_comments['comment']))

AUC: 0.959
Accuracy: 95.034% 

Confusion matrix:
[[20124   298]
 [  853  1903]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.96      0.99      0.97     20422
     Attack       0.86      0.69      0.77      2756

avg / total       0.95      0.95      0.95     23178



## SGD Classifier
* ran a grid search for parameters alpha and n_iter because it's recommended in the documentation

In [558]:
# With default values

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.decision_function(test_comments['comment']))

AUC: 0.945
Accuracy: 90.672% 

Confusion matrix:
[[19383    58]
 [ 2104  1633]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.90      1.00      0.95     19441
     Attack       0.97      0.44      0.60      3737

avg / total       0.91      0.91      0.89     23178



In [559]:
# Best with manual tuning

clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000,ngram_range = (1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'modified_huber', alpha =0.00006,n_iter=65)),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.decision_function(test_comments['comment']))

AUC: 0.942
Accuracy: 92.316% 

Confusion matrix:
[[19150   291]
 [ 1490  2247]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.93      0.99      0.96     19441
     Attack       0.89      0.60      0.72      3737

avg / total       0.92      0.92      0.92     23178



In [377]:
print("Best score: %0.3f" % gscv.best_score_)
print("Best parameters set:")
best_parameters = gscv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000,ngram_range = (1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'modified_huber', alpha = best_parameters['clf__alpha'],
                          n_iter = best_parameters['clf__n_iter'])),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.decision_function(test_comments['comment']))

Best score: 0.943
Best parameters set:
	clf__alpha: 0.0001
	clf__n_iter: 60
AUC: 0.957
Accuracy: 94.301% 

Confusion matrix:
[[20293   129]
 [ 1192  1564]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.94      0.99      0.97     20422
     Attack       0.92      0.57      0.70      2756

avg / total       0.94      0.94      0.94     23178



In [536]:
# Grid search

clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000,ngram_range = (1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'modified_huber')),
])

parameters = {
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    #'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': 10.0**-np.arange(1,7),
    #'clf__penalty': ('l2', 'elasticnet'),
    'clf__n_iter': (40, 60, 80),
}

gscv = GridSearchCV(clf,parameters)
gscv.fit(train_comments['comment'], train_comments['attack'])

Best score: 0.916
Best parameters set:
	clf__alpha: 0.0001
	clf__n_iter: 40
AUC: 0.945


ValueError: Found input variables with inconsistent numbers of samples: [23178, 9]

## Random Forest

In [266]:
# With default values

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.predict_proba(test_comments['comment'])[:, 1])

AUC: 0.940
Accuracy: 93.705% 

Confusion matrix:
[[20284   138]
 [ 1321  1435]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.94      0.99      0.97     20422
     Attack       0.91      0.52      0.66      2756

avg / total       0.94      0.94      0.93     23178



In [278]:
# After tuning

rf = RandomForestClassifier(n_estimators=50)
vect = CountVectorizer(max_features = 10000, ngram_range = (1,2), min_df=5)

clf = Pipeline([
    ('vect', vect),
    ('tfidf', TfidfTransformer(norm = 'l2')),
    ('clf', rf),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
print_classification_metrics(clf.predict_proba(test_comments['comment'])[:, 1])

AUC: 0.941
Accuracy: 93.662% 

Confusion matrix:
[[20280   142]
 [ 1327  1429]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.94      0.99      0.97     20422
     Attack       0.91      0.52      0.66      2756

avg / total       0.94      0.94      0.93     23178



## Combining features

Got worse accuracy than without adding features, extra features are always classified as 'Polite', not really sure why...

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction import DictVectorizer

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.
    
    Copied from: http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


def print_classification_metrics(probabilities):
    """Prints AUC, accuracy, confusion matrix, precision, recall, fscore, and support.
    Tweaked to work with pipeline containing FeatureUnion.
    """
    auc = roc_auc_score(test_comments['attack'], probabilities)
    predicted = clf.predict(test_comments)
    
    print "AUC: %.3f" %auc
    print "Accuracy: %(acc).3f" %{"acc": (100*accuracy_score(test_comments['attack'],predicted))} + "%","\n"
    print "Confusion matrix:\n",confusion_matrix(test_comments['attack'],predicted), "\n"
    print "Other stats:\n", classification_report(test_comments['attack'],predicted,target_names = ['Polite', 'Attack'])

In [535]:
clf = Pipeline([
    # Use FeatureUnion to combine the features from comments and sample,ns, etc.
    ('union', FeatureUnion(
        transformer_list=[
            # Pipeline for standard bag-of-words model for comments
            ('comment_bow', Pipeline([
                ('selector', ItemSelector(key='comment')),
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
            ])),
            # Pipeline for pulling ad hoc features from post's body
            ('comment_stats', Pipeline([
                ('selector', ItemSelector(key='stats')),
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
                ('tfidf', TfidfTransformer()),
            ])),

        ],

        # weight components in FeatureUnion
        transformer_weights={
            'comment_bow': 0.0,
            'comment_stats': 1.0,
        },
    )),
    ('clf', SGDClassifier()),
])

clf = clf.fit(train_comments, train_comments['attack'])
y = clf.predict(test_comments)
print(classification_report(y, test_comments['attack']))
print_classification_metrics(clf.decision_function(test_comments))

             precision    recall  f1-score   support

      False       1.00      0.84      0.91     23178
       True       0.00      0.00      0.00         0

avg / total       1.00      0.84      0.91     23178

AUC: 0.803
Accuracy: 83.877% 

Confusion matrix:
[[19441     0]
 [ 3737     0]] 

Other stats:
             precision    recall  f1-score   support

     Polite       0.84      1.00      0.91     19441
     Attack       0.00      0.00      0.00      3737

avg / total       0.70      0.84      0.77     23178



# Conclusion

#### What are the features you considered using? What features did you use in the final code?
The only features I ended up using in my code were 'comment' and 'attack', although I considered using 'logged_in', 'ns', and 'sample' as well. Ultimately, I couldn't figure out how to use those features to improve the model, so they were left out of the final code.

#### What optimizations did you add in your code, if any?
I used GridSearch on the SGDClassifier model.

#### What hyper-parameter tuning did you do?
For all models, I tried tuning a few parameters each for CountVectorizer (max_features, n_grams, min_df, stop_words) and TfidfTransformer (norm, sublinear_tf). I kept whichever setting improved the model's performance the most. For the individual classifiers, I selected the parameters that seemed most likely to improve performance for that specific model--for example, n_estimators for RandomForest and loss for SGDClassifier. I also tried a few things that didn't work, like excluding stop words (both from a premade list and using max_df), and setting class_weight='balanced' as a parameter on some models. 

#### What did you learn from the different metrics?
The model accuracy and AUC were both very helpful for getting an idea of overall performance. The confusion matrix and other stats (precision, recall, etc.) were more useful for figuring out how the model was actually classifying things; for example, most models were more likely to mistake rude comments for polite ones than vice versa, probably because the training set had more polite comments in it. 

#### Did you try cross-validation?
No :(

#### What are your best final Result Metrics? By how much is it better than the strawman figure?

|                   | strawman | final  | improvement |
| ----------------- |:--------:|:------:| -----------:|
| AUC               | 0.957    | 0.959  | 0.002       |
| Accuracy          | 94.05%   | 95.04% | 0.99%       |
| Average precision | 0.94     | 0.95   | 0.01        |
| Average recall    | 0.94     | 0.95   | 0.01        |
| Average f1-score  | 0.93     | 0.95   | 0.02        |


#### Which model gave you this performance?
My best final results metrics were for the Linear SVM model after tuning.

#### What is the most interesting thing you learned from doing the report?
I looked up the most predictive features for each model to verify that it was working correctly (using the `print_top10_features` function). The most predictive words for the attack comments were all curse words, except for the word 'you', which showed up a few times. This makes sense, since a lot of attacks are directly addressing the person being attacked. Similarly, the most predictive words for polite comments were 'thanks' and 'thank you'. It was highly predictive of both attack and polite comments (depending on the order), which is probably partly why every model did better when stop words were included.

#### What was the hardest thing to do?
As a model improved, it became very hard to get it to improve further with tuning. The Naive Bayes' model had the greatest improvement after tuning, but it started out with the worst performance. The Logistic Regression and LinearSVM models were both fairly high without tuning, and it was difficult to find a combination of parameters where either of them improved significantly.

## Other stuff (will probably delete)

In [140]:
comment_text = test_comments['comment'].values
isattack = test_comments['attack'].values
predicted

array([ 0.06007826,  0.12342553,  0.05361463, ...,  0.05755827,
        0.06621351,  0.01309482])

In [183]:
print_top10(vect, lr, train_comments['attack'])

most attack:
cock 
bastard 
gay 
penis 
cunt 
bullshit 
sucks 
idiots 
moron 
dick 
suck 
bitch 
you 
asshole 
ass 
shit 
stupid 
idiot 
fucking 
fuck


least attack:
thank you 
thank 
article 
thanks 
please 
could 
section 
agree 
but 
there 
welcome 
source 
redirect talk 
redirect 
after 
is not 
which 
discussion 
different 
if


In [45]:
isattack

array([False, False, False, ..., False, False, False], dtype=bool)

In [85]:
def correctprediction(predicted, actual):
    if(actual):
        return predicted >= .50
    else:
        return predicted < .50

In [86]:
correctprediction(predicted[0], isattack[0])

True

In [87]:
for i in range(len(predicted)):
    if not correctprediction(predicted[i], isattack[i]):
        print "Is attack: ", isattack[i], 
        print ", Predicted value: ", predicted[i]
        print "Comment text: ", comment_text[i]
        print ""

Is attack:  True , Predicted value:  0.279472441181
Comment text:  yes  complain to your rabbi and then go shoot some more defenseless arabs to take out your frustration     

Is attack:  False , Predicted value:  0.769515018621
Comment text:  god damn god damn god damn god damn god damn god damn god damn god damn god damn god damn

Is attack:  True , Predicted value:  0.126633873546
Comment text:        hey  polish slavic monkeys     you slavic monkeys can wright here all the shit what you want  but all world knews what realy you are   the trashy mutantes from lower dniepr  your   poland   have no rights to exist   it is only band of slavic trampes    vilnietis 

Is attack:  True , Predicted value:  0.111704527066
Comment text:    obviously lan3y knows more about the band than mark and tom  an  example of stupidity 

Is attack:  False , Predicted value:  0.59561657208
Comment text:        skateboard     i do not own a skateboard knob end 

Is attack:  True , Predicted value:  0.386570

In [90]:
predicted[0] < .50

True